In [1]:
import torch
import torch.nn as nn
import numpy
import nltk
import re
import string
import matplotlib.pyplot as plt

In [3]:
f = open(r"/home/kandpal/Desktop/Intro to NLP/Language Models/Dataset/shakespeare sonnets.txt","r",encoding='utf-8-sig')

words=''
for i in f:
    words += i
    
# string mein store kiya kyuki string operations karne hai baad mein
# utf-8-sig instead of just utf-8 kyuki kuch toh special characters read karne mein problem aata hai utf-8 ko

In [4]:
words = words.translate(str.maketrans('','',string.digits))
words = words.translate(str.maketrans('','',string.punctuation))
words = str(words.lower())

words = nltk.word_tokenize(words)

# digit punctuation nikalke tokenize kiya

In [5]:
vocab_size = len(set(words))
vocab = set(words)

int2word = {}
word2int = {}

for ids,unique in enumerate(vocab):
    word2int[unique] = ids
    int2word[ids] = unique
    
# yeh cell bhi self explanatory hai

In [6]:
embbsize = 256
# embbsize tera features ka dimensions hai

vocab_vector = torch.zeros(1,vocab_size)
# ek vector hai usme jo word rahega usko 1 kar dunga one hot ke liye, word represent karne ke liye

embeddings = torch.randn(vocab_size, embbsize, requires_grad=True)
# maine randomly initialised vector liya hai, requires grad true hai so uspe gradient update karunga har word ke liye

In [ ]:
class Model(nn.Module):
    def __init__(self,embbsize, vocab_size, hidden_size=1024):
        super(Model, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(embbsize, hidden_size, num_layers=2, dropout=0.5, batch_first=True)
        
        self.fc = nn.Linear(hidden_size,vocab_size)

    def forward(self, x):
        output, hidden = self.lstm(x)    
        out = self.fc(output)
        return out,hidden
    
# simple model hai 2 stacked lstm, fir ek fc jo output dega

In [ ]:
net = Model(embbsize,vocab_size).cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-6, weight_decay=1e-4, amsgrad=True)
lossfn = nn.CrossEntropyLoss() 

# self explanatory

In [ ]:
costs = []
total_loss = []
# batch_size = 56
epoch = 20

for i in range(epoch):
    
    coll_loss = 0
    
    for j,cor_word in enumerate(words):
        
        blue = vocab_vector
        # vocab vector ka copy liya because har word ke liye ek one hot banana hai, har iteration mein
        
        blue[0][word2int[cor_word]] = 1
        # jo word hai usko one kiya
        
        word_embed = torch.matmul(blue, embeddings)
        # multiply kiya so that now mere paas jo word hai uska hi embeddings aa gaya
        
        word_embed = torch.unsqueeze(word_embed,0)
        # lstm ko 1,1,inputsize chahiye hota hai na but sirf 1,inputsize hai abhi,,,, so unsqueeze
        
        if(j==len(words)-1):
            ytarget = torch.tensor(0, dtype=torch.int64)
            # yeh kuch bhi kiya hai iska solution nai samjha merko, jab last word ayega toh kya target rakhu
            
        else:
            ytarget = torch.tensor(word2int[words[j+1]], dtype = torch.int64) #everythn should be a torch tensor in pytorch
            # next word ka word2int kiya
            
        output,hidden1 = net(word_embed.cuda())   
        
        output = output.view(1,-1)   # shape is ([1,3119])
        ytarget = torch.unsqueeze(ytarget,0)   # shape is ([1]) && value is ([1386])
            
        loss = lossfn(output.cuda(),ytarget.cuda())
        costs.append(loss)
        coll_loss += loss.item()
            
        optimizer.zero_grad()
        
        loss.backward()
        # isse mera embeddings ka bhi gradients compute hona chahiye, but mai confused hoon ki embedding matrix actually update hua ya nai
            
        optimizer.step()
        # parameter update
        
    total_loss.append(coll_loss)
        
    print("epoch number", i, coll_loss)

In [ ]:
plt.plot(costs)
plt.title('Costs')

In [ ]:
plt.plot(total_loss)
plt.title('Total loss')

In [ ]:
from scipy.ndimage import gaussian_filter

cost = [float(i.detach().cpu()) for i in costs]
smooth = gaussian_filter(cost,800)
plt.plot(smooth)